# Loading a Saved Model
1. Serving a model can be via a REST API, or on a device (such as Android or iOS) or on the browser (as a Javascript). Tensorflow provides support for all of these options. See Chollet section 6.3.2, page 166 for more.
2. During long trainings it makes sense to save your model. A version of a model (with the weights and optimizer status at that time) is called 'checkpoint'. To see how to use that in Keras, see Chollet section 7.3.2, page 187.

For serving a Model with Tensorflow see also:
1. https://github.com/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb
2. https://www.tensorflow.org/tfx/tutorials/serving/rest_simple

In [1]:
import tensorflow as tf

In [2]:
# get the test MNIST data
mnist = tf.keras.datasets.mnist
_, (test_images, test_labels) = mnist.load_data()
# preprocess
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [3]:
# load the model with Keras:
test_model = tf.keras.models.load_model("./mnist_model/keras_pb/")

In [4]:
# perdict a single image
prediction = test_model.predict(test_images[0:1])

1/1 [==============================] - 0s 64ms/step


In [5]:
prediction.argmax()  # this is the same as the model at the time it was saved, as expected

7

In [6]:
test_loss, test_acc = test_model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")  # this is the same as the model at the time it was saved, as expected

313/313 [==============================] - 1s 1ms/step - loss: 0.0615 - accuracy: 0.9812
test_acc: 0.9811999797821045


In [7]:
# it is useful to benchmark the time it takes to predict a single image
# a complete benchmark would need to run the prediction multiple times
# and list the configuration (such as the CPU or GPU stypes and the versions of Python and TensorFlow)
import time

start = time.time()
prediction = test_model.predict(test_images[0:1])
end = time.time()
print(f"Time to predict a single image: {round((end - start) * 1000)} ms")

1/1 [==============================] - 0s 23ms/step
Time to predict a single image: 53 ms


### Load and Infer with Tensorflow 

In [8]:
loaded = tf.saved_model.load('./mnist_model/tf/')
print(list(loaded.signatures.keys()))

['serving_default']


In [9]:
infer = loaded.signatures["serving_default"] 

In [10]:
print(infer.structured_outputs)  # this provides the name of the output layer

{'dense_1': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_1')}


In [11]:
prediction = infer(tf.constant(test_images[0:1]))['dense_1']

In [12]:
prediction

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[2.0238337e-08, 4.3006159e-09, 2.2429526e-06, 3.1001149e-05,
        2.4654953e-12, 1.0138720e-07, 4.4970555e-12, 9.9996340e-01,
        5.6754946e-08, 3.1188131e-06]], dtype=float32)>

In [13]:
prediction.numpy().argmax()  # same as the model at the time it was saved, as expected

7

In [14]:
start = time.time()
prediction = infer(tf.constant(test_images[0:1]))['dense_1']
end = time.time()
print(f"Time to predict a single image: {round((end - start) * 1000, 1)} ms")

Time to predict a single image: 1.0 ms


In [15]:
# note that a models saved with Keras can be loaded with TensorFlow
loaded = tf.saved_model.load('./mnist_model/keras_pb/')
infer = loaded.signatures["serving_default"] 
prediction = infer(tf.constant(test_images[0:1]))['dense_1']
prediction.numpy().argmax()  # same as the model at the time it was saved, as expected

7

In [16]:
# However, loading a model saved with TensorFlow with Keras will not work:
try:
    tf.keras.models.load_model("./mnist_model/tf/")
except Exception as e:
    print(e)

Unable to create a Keras model from SavedModel at ./mnist_model/tf/. This SavedModel was exported with `tf.saved_model.save`, and lacks the Keras metadata file. Please save your Keras model by calling `model.save` or `tf.keras.models.save_model`. Note that you can still load this SavedModel with `tf.saved_model.load`.


In [17]:
# the difference between the two folders is an additional `keras_metadata.pb` file in the Keras folder

In [18]:
# h5 Keras model can't be loaded with TensorFlow
try:
    loaded = tf.saved_model.load('./mnist_model/keras_h5/model.h5')
except Exception as e:
    print(e)


SavedModel file does not exist at: ./mnist_model/keras_h5/model.h5\{saved_model.pbtxt|saved_model.pb}


In [21]:
# if only inference (but not training) is needed, the model saved without the optimizer can be loaded
loaded = tf.saved_model.load('./mnist_model/keras_pb_no_optimizer/')
infer = loaded.signatures["serving_default"] 
prediction = infer(tf.constant(test_images[0:1]))['dense_1']
prediction.numpy().argmax()  # same as the model at the time it was saved, as expected

7

In [22]:
infer.structured_input_signature

((),
 {'dense_input': TensorSpec(shape=(None, 784), dtype=tf.float32, name='dense_input')})